In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수 67: PER 71:자본금 72: 액면가
        rqField = [0, 4, 20, 67, 71, 72]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
            per = self.objRq.GetDataValue(3, i)  # per
            base = self.objRq.GetDataValue(4, i)  # base
            face = self.objRq.GetDataValue(5, i)  # 액면가
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
                listedStock *= 1000
                print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt, per, base, face)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        allcodelist = ("U001",) + codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
                
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [3]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2967, 거래소 1550, 코스닥 1416
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
A005930 319980315200000
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2967 종목)
삼성전자 상장주식수: 5,969,782,000, 시가총액 319,980,315,200,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 62,171,401,971,000
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 51,013,215,000,000
NAVER 상장주식수: 164,263,395, 시가총액 45,583,092,112,500
셀트리온 상장주식수: 134,939,385, 시가총액 41,763,739,657,500
삼성전자우 상장주식수: 822,886,700, 시가총액 38,593,386,230,000
LG화학 상장주식수: 70,592,343, 시가총액 35,790,317,901,000
삼성SDI 상장주식수:

제주항공 상장주식수: 26,356,758, 시가총액 438,840,020,700
코엔텍 상장주식수: 50,000,000, 시가총액 438,500,000,000
메디포스트 상장주식수: 15,617,842, 시가총액 436,518,683,900
한세실업 상장주식수: 40,000,000, 시가총액 434,000,000,000
오스템임플란트 상장주식수: 14,285,717, 시가총액 432,857,225,100
메가스터디교육 상장주식수: 11,821,880, 시가총액 432,680,808,000
LG전자우 상장주식수: 17,185,992, 시가총액 432,227,698,800
SK증권 상장주식수: 472,590,171, 시가총액 431,002,235,952
테스 상장주식수: 19,768,226, 시가총액 430,947,326,800
EDGC 상장주식수: 36,172,940, 시가총액 430,457,986,000
덴티움 상장주식수: 11,068,830, 시가총액 430,024,045,500
바디텍메드 상장주식수: 23,477,551, 시가총액 428,465,305,750
이녹스첨단소재 상장주식수: 9,502,493, 시가총액 427,137,060,350
현대건설기계 상장주식수: 19,700,793, 시가총액 423,567,049,500
대한해운 상장주식수: 24,427,109, 시가총액 422,588,985,700
바이넥스 상장주식수: 31,741,048, 시가총액 422,155,938,400
삼양사 상장주식수: 10,313,449, 시가총액 421,820,064,100
세방전지 상장주식수: 14,000,000, 시가총액 420,700,000,000
동화약품 상장주식수: 27,931,470, 시가총액 420,368,623,500
BGF 상장주식수: 95,716,791, 시가총액 417,803,792,715
미원에스씨 상장주식수: 5,100,000, 시가총액 417,690,000,000
한국아트라스비엑스 상장주식수: 9,150,000, 시가총액 417,240,000,00

슈프리마에이치큐 상장주식수: 10,471,840, 시가총액 50,369,550,400
대구백화점 상장주식수: 10,821,611, 시가총액 50,266,383,095
피델릭스 상장주식수: 26,299,866, 시가총액 50,232,744,060
엔케이물산 상장주식수: 83,578,428, 시가총액 50,147,056,800
코오롱인더우 상장주식수: 2,767,860, 시가총액 50,098,266,000
쎄미시스코 상장주식수: 5,637,679, 시가총액 49,837,082,360
TIGER 미국채10년선물 상장주식수: 3,900,000, 시가총액 49,705,500,000
이글루시큐리티 상장주식수: 10,996,119, 시가총액 49,702,457,880
우신시스템 상장주식수: 18,314,054, 시가총액 49,631,086,340
에스피시스템스 상장주식수: 8,145,700, 시가총액 49,607,313,000
휴스틸 상장주식수: 7,621,668, 시가총액 49,540,842,000
메디앙스 상장주식수: 11,800,000, 시가총액 49,501,000,000
제룡산업 상장주식수: 10,000,000, 시가총액 49,400,000,000
흥국 상장주식수: 12,322,696, 시가총액 49,290,784,000
미래테크놀로지 상장주식수: 6,120,788, 시가총액 49,272,343,400
하이소닉 상장주식수: 14,765,297, 시가총액 49,168,439,010
TPC 상장주식수: 13,480,276, 시가총액 48,866,000,500
NE능률 상장주식수: 16,072,037, 시가총액 48,778,632,295
오리콤 상장주식수: 11,704,050, 시가총액 48,747,368,250
우진아이엔에스 상장주식수: 7,622,000, 시가총액 48,704,580,000
인터지스 상장주식수: 29,747,874, 시가총액 48,637,773,990
오가닉티코스메틱 상장주식수: 56,925,000, 시가총액 48,500,100,000
삼성 코스피 풋

ARIRANG 단기우량채권 상장주식수: 140,000, 시가총액 7,429,800,000
TIGER 국채선물10년인버스 상장주식수: 160,000, 시가총액 7,414,400,000
KG동부제철우 상장주식수: 53,385, 시가총액 7,393,822,500
KBSTAR IT플러스 상장주식수: 500,000, 시가총액 7,360,000,000
KODEX 에너지화학 상장주식수: 700,000, 시가총액 7,357,000,000
KINDEX 스마트밸류 상장주식수: 700,000, 시가총액 7,353,500,000
크라운해태홀딩스우 상장주식수: 600,726, 시가총액 7,328,857,200
럭슬 상장주식수: 40,828,103, 시가총액 7,308,230,437
소프트센우 상장주식수: 142,857, 시가총액 7,285,707,000
키움제5호스팩 상장주식수: 3,600,000, 시가총액 7,272,000,000
KBSTAR 중국MSCI China선물(H) 상장주식수: 600,000, 시가총액 7,263,000,000
KINDEX 미국다우존스리츠(합성 H) 상장주식수: 100,000, 시가총액 7,255,000,000
이베스트스팩5호 상장주식수: 3,620,000, 시가총액 7,221,900,000
KOSEF 저PBR가중 상장주식수: 700,000, 시가총액 7,210,000,000
QV MSCI 선진국 ETN 상장주식수: 700,000, 시가총액 7,206,500,000
ARIRANG 코스닥150 상장주식수: 600,000, 시가총액 7,095,000,000
코오롱글로벌우 상장주식수: 312,803, 시가총액 7,069,347,800
흥국 S&P코리아로우볼 상장주식수: 840,000, 시가총액 7,068,600,000
ARIRANG 국채선물3년 상장주식수: 132,000, 시가총액 7,062,660,000
신영스팩5호 상장주식수: 3,505,000, 시가총액 7,062,575,000
엔에이치스팩13호 상장주식수: 3,420,000, 시가총액 7,045,200,0

In [4]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [5]:
data2[0]

('A005930', (5969782000, 297892121800000, 15.760000228881836, 778046, 100))

In [6]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    per = item[1][2]
    eps = item[1][3]
    face = item[1][4]
    
    MarketTotal.append([name, item[0], listed, markettot, per, eps, face])

In [7]:
import pandas as pd

In [8]:
format(1e+10, ",")

'10,000,000,000.0'

In [9]:
format(1e+12, ",")

'1,000,000,000,000.0'

In [10]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [11]:
len(pdMarketTotal)

2974

In [12]:
pdMarketTotal.to_csv("../data/KRX_LIST.csv", index=None)

In [13]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization,per,base,face".split(",")

In [14]:
pdMarketTotal.loc[pdMarketTotal.Code == "U001"]

,Name,Code,Stock,capitalization,per,base,face
2973,종합주가지수,U001,0,0.0,0.0,0,0


In [15]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 200선물인버스2X"]

,Name,Code,Stock,capitalization,per,base,face
102,KODEX 200선물인버스2X,A252670,320900000,2.056969e+12,0.0,0,0


In [16]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1471,KODEX 미국S&P500선물(H),A219480,5750000,8.052875e+10,0.0,0,0


In [17]:
pdMarketTotal.loc[pdMarketTotal.Name == "KODEX 미국S&P500선물(H)"]

,Name,Code,Stock,capitalization,per,base,face
1471,KODEX 미국S&P500선물(H),A219480,5750000,8.052875e+10,0.0,0,0


In [18]:
pdMarketTotal.loc[pdMarketTotal.Code == "A950140"]

,Name,Code,Stock,capitalization,per,base,face
1234,잉글우드랩,A950140,19867866,1.043063e+11,46.049999,0,0


In [19]:
pdMarketTotal.loc[pdMarketTotal.Code == "A900140"]

,Name,Code,Stock,capitalization,per,base,face
623,엘브이엠씨홀딩스,A900140,51120439,2.510014e+11,0.0,12,0


In [20]:
pdMarketTotal.loc[pdMarketTotal.Code == "A005930"]

,Name,Code,Stock,capitalization,per,base,face
0,삼성전자,A005930,5969782000,2.978921e+14,15.76,778046,100


In [21]:
pdMarketTotal.loc[pdMarketTotal.Code == "A000660"]

,Name,Code,Stock,capitalization,per,base,face
1,SK하이닉스,A000660,728002365,5.984179e+13,29.709999,3657652,5000


In [22]:
# pdMarketTotal_not_etf = pdMarketTotal.loc[(
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KODEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TIGER") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KBSTAR") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("HANARO") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ARIRANG") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KINDEX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("KOSEF") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("SMART") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("TREX") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("OTM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ATM") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETN") < 0)) &
#     (pdMarketTotal["Name"].apply(lambda x: x.find("ETF") < 0)) 
# )]

In [23]:
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.endswith("K"))]
pdMarketTotal = pdMarketTotal[pdMarketTotal.Code.apply(lambda x: not x.startswith("Q"))]

In [24]:
KOSPI = pdMarketTotal.loc[(
    (pdMarketTotal.Stock == 0)
), "Name,Code,Stock,capitalization".split(",")]

In [25]:
KOSPI["type"] = "index"

In [26]:
STOCK = pdMarketTotal.loc[(
    ((pdMarketTotal.face > 0) & 
     (pdMarketTotal.base > 0) & 
     (pdMarketTotal.per > 0) & 
     (pdMarketTotal.Stock > 1e+7) &
     (pdMarketTotal.capitalization > 1e+12))
), "Name,Code,Stock,capitalization".split(",")]

In [27]:
STOCK["type"] = "stock"

In [28]:
ETF = pdMarketTotal.loc[(
    ((pdMarketTotal.face == 0) & 
     (pdMarketTotal.base == 0) & 
     (pdMarketTotal.per == 0) & 
     (pdMarketTotal.capitalization > 1e+10))
), "Name,Code,Stock,capitalization".split(",")]

In [29]:
ETF["type"] = 'etf'

In [30]:
concat = pd.concat([KOSPI, STOCK, ETF])

In [31]:
concat

,Name,Code,Stock,capitalization,type
2973,종합주가지수,U001,0,0.000000e+00,index
0,삼성전자,A005930,5969782000,2.978921e+14,stock
1,SK하이닉스,A000660,728002365,5.984179e+13,stock
2,삼성바이오로직스,A207940,66165000,3.956667e+13,stock
4,NAVER,A035420,164263395,3.490597e+13,stock
...,...,...,...,...,...
2611,KODEX 헬스케어,A266420,700000,1.038100e+10,etf
2615,HANARO e커머스,A322400,950000,1.026950e+10,etf
2618,KODEX TRF5050,A329660,1000000,1.023000e+10,etf
2619,KODEX TRF7030,A329670,1000000,1.021000e+10,etf


In [40]:
concat.to_csv("../data/SELECT_LIST.csv", index=None)